In [3]:
import pandas as pd
from scipy.interpolate import PchipInterpolator
import numpy as np

In [4]:
df = pd.read_csv('BMED_DATA.csv')

df_ag = pd.DataFrame()

# exp 값을 0부터 시작해서 실험별로 연속적으로 증가하도록 매핑
unique_exps = sorted(df['exp'].unique())
exp_map = {old_exp: new_exp for new_exp, old_exp in enumerate(unique_exps)}

for exp_num in unique_exps:
    exp_data = df[df['exp'] == exp_num]

    # For each experiment, generate new time points at 0.25 intervals up to the end time.
    new_times = np.arange(0, exp_data['time'].max() + 0.25, 0.25)

    # The columns to be interpolated
    columns_to_interpolate = ['VF', 'VA', 'VB', 'CFLA', 'CALA', 'CFK','CBK', 'I']

    # exp 값을 0부터 시작해서 연속적으로 증가하도록 저장
    temp_df = pd.DataFrame({
        'exp': exp_map[exp_num],
        'V': exp_data['V'].iloc[0],
        'E': exp_data['E'].iloc[0],
        't': new_times
    })

    # apply the monotonic cubic interpolation for each column
    for col in columns_to_interpolate:
        interpolator = PchipInterpolator(exp_data['time'], exp_data[col])
        temp_df[col] = interpolator(new_times)
    
    # add the results to main dataframe
    if df_ag.empty:
        df_ag = temp_df
    else:
        df_ag = pd.concat([df_ag, temp_df], ignore_index=True)

In [5]:
# save CSV file
df_ag.to_csv('BMED_DATA_AG.csv', index=False)

In [6]:
df_ag

,exp,V,E,t,VF,VA,VB,CFLA,CALA,CFK,CBK,I
0,0,20,0.25,0.00,1.000000,1.000000,1.000000,0.466577,0.000180,1.032157,0.007676,0.000000
1,0,20,0.25,0.25,1.001699,0.998894,0.999175,0.465729,0.000196,1.029884,0.008503,0.025000
2,0,20,0.25,0.50,1.006196,0.996620,0.996966,0.463613,0.000238,1.023399,0.010929,0.050000
3,0,20,0.25,0.75,1.012595,0.993535,0.993774,0.460647,0.000299,1.013204,0.014869,0.075000
4,0,20,0.25,1.00,1.020000,0.990000,0.990000,0.457245,0.000370,0.999798,0.020242,0.100000
...,...,...,...,...,...,...,...,...,...,...,...,...
885,35,35,0.50,6.00,0.282901,1.443299,2.273113,0.253424,2.204487,0.041429,1.184012,3.404974
886,35,35,0.50,6.25,0.263436,1.443299,2.291290,0.168860,2.222514,0.025455,1.181657,2.937112
887,35,35,0.50,6.50,0.248414,1.443299,2.306515,0.104276,2.235851,0.013400,1.176642,2.319837
888,35,35,0.50,6.75,0.239264,1.443299,2.316642,0.065264,2.243695,0.006190,1.169461,1.588445
